In [1]:
import sys
sys.path.insert(0,'..')

In [8]:
import torch
from torch import nn

from tqdm import tqdm


from train import train, Losses
import priors
import encoders
import positional_encodings
import utils
import bar_distribution


In [19]:
mykwargs = \
{'extra_prior_kwargs_dict': {'num_features': 28*28, 'fuse_x_y': False, 'num_outputs':5, 'only_train_for_last_idx': True,},
 'bptt': 5*5+1,
'nlayers': 6,
 'dropout': 0.0, 'steps_per_epoch': 100,
 'batch_size': 1000}
mnist_jobs_5shot_pi_even_longer_running = [
    train(priors.stroke.DataLoader, Losses.ce, enc, emsize=emsize, nhead=nhead, warmup_epochs=warmup_epochs, nhid=nhid, y_encoder_generator=encoders.get_Canonical(5), lr=lr, epochs=epochs, single_eval_pos_gen=mykwargs['bptt']-1, **mykwargs)
    for enc in [encoders.Linear] for emsize in [1024] for nhead in [4] for nhid in [emsize*2] for warmup_epochs in [5]
    for lr in [.0001,]#.00001,.000001]
    for epochs in [1,]#64,128,256]
    for _ in range(1)]

Using cpu:0 device
DataLoader.__dict__ {'num_steps': 1, 'fuse_x_y': False, 'get_batch_kwargs': {'batch_size': 1000, 'seq_len': 26, 'num_features': 784, 'num_outputs': 5, 'only_train_for_last_idx': True}, 'num_features': 784, 'num_outputs': 5}
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 130.12s | mean loss  1.63 | pos losses   nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan, 1.63, lr 0.0 data time  6.38 step time 123.71 forward time 44.98
-----------------------------------------------------------------------------------------


In [20]:

@torch.inference_mode()
def get_acc(finetuned_model, eval_pos, device='cpu', steps=100, train_mode=False, **mykwargs):
    finetuned_model.to(device)
    finetuned_model.eval()

    t_dl = priors.omniglot.DataLoader(steps, batch_size=1000, seq_len=mykwargs['bptt'], train=train_mode,
                                      **mykwargs['extra_prior_kwargs_dict'])

    ps = []
    ys = []
    for x, y in tqdm(t_dl):
        p = finetuned_model(tuple(e.to(device) for e in x), single_eval_pos=eval_pos)
        ps.append(p)
        ys.append(y)

    ps = torch.cat(ps, 1)
    ys = torch.cat(ys, 1)

    def acc(ps, ys):
        return (ps.argmax(-1) == ys.to(ps.device)).float().mean()

    a = acc(ps[eval_pos], ys[eval_pos]).cpu()
    print(a.item())
    return a


def train_and_eval(*args, **kwargs):
    r = train(*args, **kwargs)
    model = r[-1]
    acc = get_acc(model, -1, device='cuda:0', **kwargs).cpu()
    model.to('cpu')
    return [acc]

def pretrain_and_eval(extra_prior_kwargs_dict_eval,*args, **kwargs):
    r = train(*args, **kwargs)
    model = r[-1]
    kwargs['extra_prior_kwargs_dict'] = extra_prior_kwargs_dict_eval
    acc = get_acc(model, -1, device='cuda:0', **kwargs).cpu()
    model.to('cpu')
    return r, acc

In [22]:
pretrained_model = mnist_jobs_5shot_pi_even_longer_running[0][-1]

In [25]:

emsize = 1024
# mnist_jobs_5shot_pi[20].result()[-1].state_dict()
mykwargs = \
    {'bptt': 5 * 5 + 1,
     'nlayers': 6,
     'nhead': 4, 'emsize': emsize,
     'encoder_generator': encoders.Linear, 'nhid': emsize * 2}
results = train_and_eval(priors.omniglot.DataLoader, Losses.ce, y_encoder_generator=encoders.get_Canonical(5),
                   load_weights_from_this_state_dict=pretrained_model.state_dict(), epochs=32, lr=.00001, dropout=0.,
                   single_eval_pos_gen=mykwargs['bptt'] - 1,
                   extra_prior_kwargs_dict={'num_features': 28 * 28, 'fuse_x_y': False, 'num_outputs': 5,
                                            'translations': True, 'jonas_style': True},
                   batch_size=100, steps_per_epoch=200, **mykwargs)



Using cpu:0 device
== Downloading https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip
== Unzip from omniglot/raw/images_background.zip to omniglot/processed
== Downloading https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip
== Unzip from omniglot/raw/images_evaluation.zip to omniglot/processed
Download finished.
== Found 32460 items 
== Found 1623 classes
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 2411.42s | mean loss  1.63 | pos losses   nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan, 1.63, lr 0.0 data time  2.08 step time 10.09 forward time  3.42
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 2241.11

-----------------------------------------------------------------------------------------
| end of epoch  18 | time: 2138.53s | mean loss  1.61 | pos losses   nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan, 1.61, lr 7.703204087277989e-06 data time  1.30 step time  9.34 forward time  3.17
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  19 | time: 2160.68s | mean loss  1.61 | pos losses   nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan, 1.61, lr 7.0770750650094335e-06 data time  1.27 step time  9.34 forward time  3.14
-----------------------------------------------------------------------------------------
---------------------------------------

KeyboardInterrupt: 